Load the dataset and the correct answer

In [1]:
!pip install transformers
!pip install datasets
!pip install librosa
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets==3.6.0 transformers accelerate soundfile evaluate jiwer tensorboard gradio tensorflow
!conda install -c conda-forge ffmpeg -y
!pip install torchcodec
!pip install sentence_transformers
!pip install tf-keras
!pip install evaluate
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio] accelerate

/home/zeus/miniconda3/lib/python3.10/site-packages/conda/base/context.py:211: FutureWarning: Adding 'defaults' to channel list implicitly is deprecated and will be removed in 25.9. 

To remove this warning, please choose a default channel explicitly with conda's regular configuration system, e.g. by adding 'defaults' to the list of channels:

  conda config --add channels defaults

For more information see https://docs.conda.io/projects/conda/en/stable/user-guide/configuration/use-condarc.html

  deprecated.topic(
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.5.1
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

zsh:1: no matches found: datasets[audio]


In [2]:
import os
audio_folder_path='/teamspace/studios/this_studio/sentence_audios'
sentence_audio_file=[]
unique_location=[]
for file in os.listdir(audio_folder_path):
    if file.lower().endswith('.wav'):
        sentence_audio_file.append(os.path.join(audio_folder_path, file))
        unique_location.append(file.replace('.wav', '').replace(', ', ''))
sentence_audio_file

['/teamspace/studios/this_studio/sentence_audios/上水, 彩蒲苑.wav',
 '/teamspace/studios/this_studio/sentence_audios/北角, 百福花園.wav',
 '/teamspace/studios/this_studio/sentence_audios/屯門, 澤豐花園.wav',
 '/teamspace/studios/this_studio/sentence_audios/沙田, 中文大學.wav',
 '/teamspace/studios/this_studio/sentence_audios/沙田, 富安花園.wav',
 '/teamspace/studios/this_studio/sentence_audios/粉嶺, 祥華邨.wav',
 '/teamspace/studios/this_studio/sentence_audios/葵涌, 清麗苑.wav',
 '/teamspace/studios/this_studio/sentence_audios/薄扶林, 碧瑤灣.wav',
 '/teamspace/studios/this_studio/sentence_audios/西灣河, 寶蕊樓.wav',
 '/teamspace/studios/this_studio/sentence_audios/鰂魚涌, 寶峰園.wav']

In [3]:
unique_location

['上水彩蒲苑',
 '北角百福花園',
 '屯門澤豐花園',
 '沙田中文大學',
 '沙田富安花園',
 '粉嶺祥華邨',
 '葵涌清麗苑',
 '薄扶林碧瑤灣',
 '西灣河寶蕊樓',
 '鰂魚涌寶峰園']

In [4]:
correct_answer_path='/teamspace/studios/this_studio/scripts/script.txt'
correct_answer=[]
with open(correct_answer_path, 'r', encoding='utf-8') as file:
    for line in file:
        correct_answer.append(line.strip())
correct_answer

['上水彩蒲苑預約拆除石油氣錶',
 '沙田富安花園預約抄錶',
 '西灣河寶蕊樓預約檢查及維修煮食爐',
 '預約屯門澤豐花園叫氣',
 '粉嶺祥華邨預約爐具安裝煮食爐',
 '鰂魚涌寶峰園預約檢查及維修其他',
 '葵涌清麗苑預約安裝石油氣錶',
 '我既屋企係薄扶林碧瑤灣',
 '北角百福花園預約檢查及維修喉管',
 '我係沙田中文大學讀緊書']

上載large turbo v3並讀取輸出結果

In [15]:
import torch
# from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
import glob
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
device

'cuda:0'

In [6]:
!pip install transformers

In [7]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
model_id = "openai/whisper-large-v3-turbo"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)
# forced_decoder_ids = processor.get_decoder_prompt_ids(language="zh", task="transcribe", "repetition_penalty": 1.5,)
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    generate_kwargs={
        # "forced_decoder_ids": forced_decoder_ids
        "language": "zh",  # 指定中文輸出
        "task": "transcribe",
        "temperature": 0.0,  # 關閉隨機性
        "repetition_penalty": 1.5,  # 抑制重複
        # "no_repeat_ngram_size": 1,
    },
)

2025-08-12 10:41:05.280344: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-12 10:41:05.303857: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754995265.329162   60218 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754995265.337817   60218 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754995265.361257   60218 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [9]:
predicted_result=[]
for file in sentence_audio_file:
    result = pipe(file)
    print(result['text'])
    predicted_result.append(result['text'])

`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.


上水彩蒲丸欲約拆除石油氣表
北角百福花園預約檢查及維修喉管
預約屯門閘風花園叫戲
我是沙田中文大學正在讀書
沙田庫安花園預約抽表
粉領長華村預約爐具安裝、煮食爐
葵聰清麗院預約安裝石油氣表
我的家是薄乎臨壁搖灣
西灣河保瑞樓預約檢查及維修主食爐
質疑中補風源預約檢查及維修其他


In [10]:
from evaluate import load
wer = load("wer")
wer_score = wer.compute(predictions=predicted_result, references=correct_answer)
print(wer_score)

1.0


加入BC Whisper進行測試

In [11]:
unique_location

['上水彩蒲苑',
 '北角百福花園',
 '屯門澤豐花園',
 '沙田中文大學',
 '沙田富安花園',
 '粉嶺祥華邨',
 '葵涌清麗苑',
 '薄扶林碧瑤灣',
 '西灣河寶蕊樓',
 '鰂魚涌寶峰園']

In [12]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from transformers import LogitsProcessor
import numpy as np

# 初始化模型和設備
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if device == "cuda" else torch.float32

model_id = "openai/whisper-large-v3-turbo"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)

# --- 核心改動：CB Whisper 實現 ---
# 1. 定義偏置詞表和地名列表 (根據您提供的標準答案)
bias_terms = {
    # 地名修正 (權重最高)
    "彩蒲苑": 20.0, "富安花園": 20.0, "寶蕊樓": 20.0,
    "澤豐花園": 20.0, "祥華邨": 20.0, "寶峰園": 20.0,
    "清麗苑": 20.0, "碧瑤灣": 20.0, "百福花園": 20.0,
    
    # 粵語專用字
    "錶": 15.0, "喺": 12.0, "咗": 12.0, "嘅": 10.0,
    "讀緊": 10.0, "屋企": 10.0, "預約": 8.0,
    
    # 行業詞彙
    "煮食爐": 15.0, "石油氣": 10.0, "喉管": 10.0
}

place_names = [
    '上水彩蒲苑', '沙田富安花園', '西灣河寶蕊樓',
    '屯門澤豐花園', '粉嶺祥華邨', '鰂魚涌寶峰園',
    '葵涌清麗苑', '薄扶林碧瑤灣', '北角百福花園'
]

# 2. 自定義Logits Processor
class CantoneseLogitsProcessor(LogitsProcessor):
    def __init__(self, processor, bias_terms, place_names):
        self.processor = processor
        self.bias_terms = bias_terms
        self.place_names = place_names

    def __call__(self, input_ids, scores):
        # 獲取當前已生成的文本
        decoded_text = self.processor.decode(input_ids[0], skip_special_tokens=True)
        print(decoded_text)
        # (A) 靜態詞表偏置
        for term, bias in self.bias_terms.items():
            term_ids = self.processor.tokenizer.encode(term, add_special_tokens=False)
            for term_id in term_ids:
                scores[0, term_id] += bias

        # (B) 地名模糊匹配修正
        for place in self.place_names:
            if place[:2] in decoded_text:  # 部分匹配觸發
                place_ids = self.processor.tokenizer.encode(place, add_special_tokens=False)
                for pid in place_ids:
                    scores[0, pid] += 25.0  # 極高權重

        # (C) 語法規則：檢測"預約"後應接動詞
        if "預約" in decoded_text[-5:]:
            for verb in ["拆除", "安裝", "檢查", "維修", "抄"]:
                verb_ids = self.processor.tokenizer.encode(verb, add_special_tokens=False)
                for vid in verb_ids:
                    scores[0, vid] += 10.0

        return scores

# 3. 初始化Pipeline並注入Processor
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    generate_kwargs={
        "language": "zh",
        "task": "transcribe",
        "temperature": 0.0,
        "repetition_penalty": 1.5,
        "logits_processor": [CantoneseLogitsProcessor(processor, bias_terms, place_names)]
    }
)

# # 4. 後處理函數 (基於規則修正)
# def cantonese_postprocess(text):
#     replacements = {
#         r"石油氣表": "石油氣錶",
#         r"正在讀書": "讀緊書",
#         r"我的家是": "我屋企係",
#         r"預約抽表": "預約抄錶"
#     }
#     for pattern, repl in replacements.items():
#         text = re.sub(pattern, repl, text)
#     return text

# --- 使用示例 ---
for file in sentence_audio_file[:1]:
    result = pipe(file)
    # final_text = cantonese_postprocess(result["text"])
    # print(result["text"])

Device set to use cuda



�
園
園�
園園
園園�
園園園
園園園�
園園園園
園園園園�
園園園園園
園園園園園�
園園園園園園
園園園園園園�
園園園園園園園
園園園園園園園�
園園園園園園園園
園園園園園園園園�
園園園園園園園園園
園園園園園園園園園�
園園園園園園園園園園
園園園園園園園園園園�
園園園園園園園園園園園
園園園園園園園園園園園�
園園園園園園園園園園園園
園園園園園園園園園園園園�
園園園園園園園園園園園園園
園園園園園園園園園園園園園�
園園園園園園園園園園園園園園
園園園園園園園園園園園園園園�
園園園園園園園園園園園園園園園
園園園園園園園園園園園園園園園�
園園園園園園園園園園園園園園園園
園園園園園園園園園園園園園園園園�
園園園園園園園園園園園園園園園園園
園園園園園園園園園園園園園園園園園�
園園園園園園園園園園園園園園園園園園
園園園園園園園園園園園園園園園園園園�
園園園園園園園園園園園園園園園園園園園
園園園園園園園園園園園園園園園園園園園�
園園園園園園園園園園園園園園園園園園園園
園園園園園園園園園園園園園園園園園園園園�
園園園園園園園園園園園園園園園園園園園園園
園園園園園園園園園園園園園園園園園園園園園�
園園園園園園園園園園園園園園園園園園園園園園
園園園園園園園園園園園園園園園園園園園園園園�
園園園園園園園園園園園園園園園園園園園園園園園
園園園園園園園園園園園園園園園園園園園園園園園�
園園園園園園園園園園園園園園園園園園園園園園園園
園園園園園園園園園園園園園園園園園園園園園園園園�
園園園園園園園園園園園園園園園園園園園園園園園園園
園園園園園園園園園園園園園園園園園園園園園園園園園�
園園園園園園園園園園園園園園園園園園園園園園園園園園
園園園園園園園園園園園園園園園園園園園園園園園園園園�
園園園園園園園園園園園園園園園園園園園園園園園園園園園
園園園園園園園園園園園園園園園園園園園園園園園園園園園�
園園園園園園園園園園園園園園園園園園園園園園園園園園園園
園園園園園園園園園園園園園園園園園園園園園園園園園園園園�
園園園園園園園園園園園園園園園園園園園園園園園園園園園園園
園園園園園園園園園園園園園園園園園園園園園園園園園園園園園�
園園園園園園園園園園園園園園園園園園園園園園園園園園園園園園
園園園園園園園園園

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Dict, Tuple, Optional
import numpy as np
from transformers import WhisperModel, WhisperProcessor
import librosa
from dataclasses import dataclass

2025-08-13 02:55:43.036434: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-13 02:55:43.064479: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755053743.088685   21175 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755053743.098023   21175 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755053743.119440   21175 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
@dataclass
class KeywordSpottingConfig:
    """Configuration for keyword spotting module"""
    cnn_channels: List[int] = None
    kernel_sizes: List[int] = None
    pooling_sizes: List[int] = None
    dropout_rate: float = 0.3
    detection_threshold: float = 0.7
    max_keywords_per_utterance: int = 5
    def __post_init__(self):
        if self.cnn_channels is None:
            self.cnn_channels = [256, 512, 512, 256]
        if self.kernel_sizes is None:
            self.kernel_sizes = [3, 3, 3, 3]
        if self.pooling_sizes is None:
            self.pooling_sizes = [2, 2, 2, 2]

In [3]:
class KeywordSpottingModule(nn.Module):
    def __init__(self,input_dim: int,num_keywords: int,config: KeywordSpottingConfig):
        super().__init__()
        self.config = config
        self.num_keywords = num_keywords
        layers = []
        in_channels = 1
        for i, (out_channels, kernel_size, pool_size) in enumerate(zip(config.cnn_channels, config.kernel_sizes, config.pooling_sizes)):
            layers.extend([
                nn.Conv1d(in_channels, out_channels, kernel_size, padding=kernel_size//2),
                nn.BatchNorm1d(out_channels),
                nn.ReLU(),
                nn.MaxPool1d(pool_size),
                nn.Dropout(config.dropout_rate)
            ])
            in_channels = out_channels
        self.cnn_layers = nn.Sequential(*layers)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Linear(config.cnn_channels[-1], num_keywords)
        self.attention = nn.MultiheadAttention(
            embed_dim=input_dim,
            num_heads=8,
            dropout=config.dropout_rate
        )
    def forward(self, encoder_hidden_states: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Forward pass for keyword spotting
        Args:
            encoder_hidden_states: [batch_size, seq_len, hidden_dim]
        Returns:
            keyword_probs: [batch_size, num_keywords]
            attention_weights: [batch_size, seq_len]
        """
        batch_size, seq_len, hidden_dim = encoder_hidden_states.shape
        attended_features, attention_weights = self.attention(
            encoder_hidden_states.transpose(0, 1),  # [seq_len, batch_size, hidden_dim]
            encoder_hidden_states.transpose(0, 1),
            encoder_hidden_states.transpose(0, 1)
        )
        attended_features = attended_features.transpose(0, 1)
        cnn_input = attended_features.reshape(batch_size, 1, -1)
        cnn_output = self.cnn_layers(cnn_input)
        pooled_features = self.global_pool(cnn_output).squeeze(-1)
        keyword_probs = torch.sigmoid(self.classifier(pooled_features))
        attention_weights = attention_weights.mean(dim=1)
        return keyword_probs, attention_weights

In [4]:
class TTSKeywordGenerator:
    """Generate synthetic audio for keyword training using TTS"""
    def __init__(self, tts_model_name: str = "microsoft/speecht5_tts"):
        self.tts_model = None
        self.sample_rate = 16000
    def generate_keyword_audio(self, keyword: str, num_samples: int = 10,contexts: List[str] = None) -> List[np.ndarray]:
        """
        Generate synthetic audio samples for a keyword
        Args:
            keyword: Target keyword
            num_samples: Number of audio samples to generate
            contexts: Optional context sentences
        Returns:
            List of audio arrays
        """
        if contexts is None:
            contexts = [
                f"The keyword is {keyword}.",
                f"Please recognize {keyword} correctly.",
                f"I mentioned {keyword} in my speech.",
                f"The term {keyword} is important.",
                f"Can you hear {keyword} clearly?"
            ]
        audio_samples = []
        for i in range(num_samples):
            context = contexts[i % len(contexts)]
            duration = len(context.split()) * 0.3  # Rough estimate
            num_samples_audio = int(duration * self.sample_rate)
            audio = np.random.randn(num_samples_audio) * 0.1  # Placeholder
            audio_samples.append(audio)
        return audio_samples
    def create_training_dataset(self,keywords: List[str],samples_per_keyword: int = 20) -> Tuple[List[np.ndarray], List[int]]:
        """
        Create training dataset for keyword spotting
        Returns:
            audio_samples: List of audio arrays
            labels: List of keyword indices
        """
        all_audio = []
        all_labels = []
        for keyword_idx, keyword in enumerate(keywords):
            keyword_audio = self.generate_keyword_audio(
                keyword, 
                num_samples=samples_per_keyword
            )
            all_audio.extend(keyword_audio)
            all_labels.extend([keyword_idx] * len(keyword_audio))
        return all_audio, all_labels

In [5]:
class PromptGenerator:
    """Generate contextual prompts based on detected keywords"""
    def __init__(self, keywords: List[str]):
        self.keywords = keywords
        self.keyword_to_idx = {kw: idx for idx, kw in enumerate(keywords)}
    def generate_prompt(self,detected_keywords: List[int],confidence_scores: List[float]) -> str:
        """
        Generate contextual prompt based on detected keywords
        Args:
            detected_keywords: List of detected keyword indices
            confidence_scores: Confidence scores for each detection
        Returns:
            Generated prompt string
        """
        if not detected_keywords:
            return ""
        keyword_conf_pairs = list(zip(detected_keywords, confidence_scores))
        keyword_conf_pairs.sort(key=lambda x: x[1], reverse=True)
        prompt_keywords = []
        for keyword_idx, conf in keyword_conf_pairs[:3]:  # Top 3 keywords
            if conf > 0.5:  # Confidence threshold
                prompt_keywords.append(self.keywords[keyword_idx])
        if not prompt_keywords:
            return ""
        if len(prompt_keywords) == 1:
            prompt = f"The following transcript contains the term {prompt_keywords[0]}."
        elif len(prompt_keywords) == 2:
            prompt = f"The following transcript contains {prompt_keywords[0]} and {prompt_keywords[1]}."
        else:
            prompt = f"The following transcript contains {', '.join(prompt_keywords[:-1])}, and {prompt_keywords[-1]}."
        return prompt

In [8]:
class CBWhisperSystem:
    """Complete CB-Whisper system implementation"""
    def __init__(self, whisper_model_name: str = "openai/whisper-small",keywords: List[str] = None,config: KeywordSpottingConfig = None, device: str = "cuda" if torch.cuda.is_available() else "cpu"):
        self.device = device
        self.processor = WhisperProcessor.from_pretrained(whisper_model_name)
        self.whisper_model = WhisperModel.from_pretrained(whisper_model_name).to(self.device)
        self.keywords = keywords or []
        self.config = config or KeywordSpottingConfig()
        if self.keywords:
            self.kws_module = KeywordSpottingModule(
                input_dim=self.whisper_model.config.d_model,
                num_keywords=len(self.keywords),
                config=self.config
            ).to(self.device)
            self.prompt_generator = PromptGenerator(self.keywords)
            self.tts_generator = TTSKeywordGenerator()
        self.is_trained = False
    def train_keyword_spotting(self, num_epochs: int = 50,batch_size: int = 16,learning_rate: float = 1e-4):
        """
        Train the keyword spotting module
        """
        if not self.keywords:
            raise ValueError("No keywords provided for training")
        print("Generating synthetic training data...")
        audio_samples, labels = self.tts_generator.create_training_dataset(
            self.keywords, 
            samples_per_keyword=50
        )
        train_features = []
        train_labels = []
        for audio, label in zip(audio_samples, labels):
            inputs = self.processor(
                audio, 
                sampling_rate=16000, 
                return_tensors="pt"
            )
            with torch.no_grad():
                input_features = inputs.input_features.to(self.device)
                encoder_outputs = self.whisper_model.encoder(input_features)
                features = encoder_outputs.last_hidden_state.cpu()
            train_features.append(features.squeeze(0))
            label_vector = torch.zeros(len(self.keywords))
            label_vector[label] = 1.0
            train_labels.append(label_vector)
        optimizer = torch.optim.Adam(self.kws_module.parameters(), lr=learning_rate)
        criterion = nn.BCELoss()
        self.kws_module.train()
        for epoch in range(num_epochs):
            total_loss = 0.0
            num_batches = 0
            for i in range(0, len(train_features), batch_size):
                batch_features = train_features[i:i+batch_size]
                batch_labels = train_labels[i:i+batch_size]
                if len(batch_features) < batch_size:
                    continue
                max_len = max(f.shape[0] for f in batch_features)
                padded_features = []
                for features in batch_features:
                    if features.shape[0] < max_len:
                        padding = torch.zeros(
                            max_len - features.shape[0], 
                            features.shape[1]
                        )
                        features = torch.cat([features, padding], dim=0)
                    padded_features.append(features)
                batch_features_tensor = torch.stack(padded_features).to(self.device)
                batch_labels_tensor = torch.stack(batch_labels).to(self.device)
                optimizer.zero_grad()
                keyword_probs, _ = self.kws_module(batch_features_tensor)
                loss = criterion(keyword_probs, batch_labels_tensor)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                num_batches += 1
            avg_loss = total_loss / num_batches if num_batches > 0 else 0
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
        self.is_trained = True
        print("Keyword spotting training completed!")
    def transcribe_with_keyword_spotting(self, audio_path: str) -> Dict:
        """
        Transcribe audio with keyword spotting enhancement
        Returns:
            Dictionary containing transcription and keyword detection results
        """
        if not self.is_trained and self.keywords:
            print("Warning: Keyword spotting module not trained. Training now...")
            self.train_keyword_spotting()
        audio, sr = librosa.load(audio_path, sr=16000)
        inputs = self.processor(audio, sampling_rate=16000, return_tensors="pt")
        input_features = inputs.input_features.to(self.device)
        with torch.no_grad():
            encoder_outputs = self.whisper_model.encoder(input_features)
            encoder_hidden_states = encoder_outputs.last_hidden_state
        detected_keywords = []
        keyword_confidences = []
        prompt = ""
        if self.keywords and self.is_trained:
            self.kws_module.eval()
            with torch.no_grad():
                keyword_probs, attention_weights = self.kws_module(encoder_hidden_states)
                keyword_probs = keyword_probs.cpu()
                for i, prob in enumerate(keyword_probs.squeeze()):
                    if prob > self.config.detection_threshold:
                        detected_keywords.append(i)
                        keyword_confidences.append(prob.item())
                if detected_keywords:
                    prompt = self.prompt_generator.generate_prompt(
                        detected_keywords, keyword_confidences
                    )
        if prompt:
            transcription_inputs = self.processor(
                audio, 
                sampling_rate=16000, 
                return_tensors="pt"
            )
            input_features = transcription_inputs.input_features.to(self.device)
            generated_ids = self.whisper_model.generate(input_features)
        else:
            generated_ids = self.whisper_model.generate(input_features)
        transcription = self.processor.batch_decode(
            generated_ids, 
            skip_special_tokens=True
        )[0]
        detected_keyword_names = [
            self.keywords[idx] for idx in detected_keywords
        ]
        return {
            "transcription": transcription,
            "detected_keywords": detected_keyword_names,
            "keyword_confidences": keyword_confidences,
            "generated_prompt": prompt,
            "attention_weights": attention_weights.squeeze().numpy() if self.keywords else None
        }

def demonstrate_cb_whisper():
    # """
    #  Demonstrate CB-Whisper implementation
    # """
    """
    Demonstrate CB-Whisper implementation on GPU
    """
    keywords = [
        "OpenAI", "Microsoft", "Google", "Amazon", "Apple",
        "TensorFlow", "PyTorch", "Kubernetes", "Docker",
        "San Francisco", "New York", "Seattle", "Boston"
    ]
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    # Initialize CB-Whisper system
    cb_whisper = CBWhisperSystem(
        whisper_model_name="openai/whisper-small",
        keywords=keywords,
        config=KeywordSpottingConfig(detection_threshold=0.6),
        device=device
    )
    print("Training keyword spotting module...")
    cb_whisper.train_keyword_spotting(num_epochs=10, batch_size=4)
    return cb_whisper


In [9]:
if __name__ == "__main__":
    system = demonstrate_cb_whisper()
    print("CB-Whisper system initialized and trained successfully")

Using device: cuda


Training keyword spotting module...
Generating synthetic training data...


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.39 GiB. GPU 0 has a total capacity of 21.96 GiB of which 2.37 GiB is free. Process 68393 has 19.59 GiB memory in use. Of the allocated memory 16.88 GiB is allocated by PyTorch, and 2.48 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)